In [9]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [10]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'Server3'

#[site1,site2,site3] = fablib.get_random_sites(count=3)
site1 = 'UCSD'
site2 = 'DALL'
print(f"Sites: {site1},{site2}")

# Hosts
h1_name = "h1"
h2_name = "h2"

h1_subnet=IPv4Network('192.168.1.0/24')
h1_addr=IPv4Address('192.168.1.2')

h2_subnet=IPv4Network('192.168.2.0/24')
h2_addr=IPv4Address('192.168.2.2')

host_cores = 8
host_ram = 16
host_disk = 500

net_s1_s2_name = 'net_s1_s2'

# All node properties
image = 'default_ubuntu_20'

Sites: UCSD,DALL


In [11]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    

    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h1_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h2_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[h1_iface, h2_iface])
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


Retry: 8, Time: 228 sec


ID,f6534eec-1dc4-447b-a92b-b657920ddce4
Name,Server3
Lease Expiration (UTC),2023-08-30 18:14:06 +0000
Lease Start (UTC),2023-08-29 18:14:07 +0000
Project ID,527832fc-c273-4254-b988-16e5c2923bf9
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
05957811-187a-43c0-8ad6-c607cc0c12bb,h1,8,16,500,default_ubuntu_20,qcow2,ucsd-w1.fabric-testbed.net,UCSD,ubuntu,132.249.252.188,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.188,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1ab103cf-5aad-4add-b4ac-286d6f2e2511,h2,8,16,500,default_ubuntu_20,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:feba:af78,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:feba:af78,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
496ec557-4baa-4a5c-9204-0b40675a5b9d,net_s1_s2,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
h1-h1_nic-p1,p1,h1,net_s1_s2,100,config,,02:74:7A:C2:0A:63,ens7,ens7,None,6
h2-h2_nic-p1,p1,h2,net_s1_s2,100,config,,06:03:79:10:FC:85,ens7,ens7,None,4



Time to print interfaces 231 seconds


In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Print the Interfaces 
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
config_threads = {}

In [ ]:
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/" 

try:
    h1 = slice.get_node(name=h1_name)        
    if type(ip_address(h1.get_management_ip())) is IPv6Address:
        h1.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
    if type(ip_address(h1.get_management_ip())) is IPv4Address:
        h1.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 8.8.8.8 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 8.8.4.4 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 1.1.1.1 >> /etc/resolv.conf'")
        
    #if type(ip_address(h1.get_management_ip())) is IPv6Address:
    #h1.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
       
    h1_os_iface = h1.get_interface(network_name=net_h1_name)
    h1_os_iface.ip_addr_add(addr=h1_addr, subnet=h1_subnet)
    h1_config_thread = h1.execute_thread(host_config_script)
    config_threads[h1] = h1_config_thread

    
    h2 = slice.get_node(name=h2_name)
    #if type(ip_address(h2.get_management_ip())) is IPv6Address:
    h2.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h2_os_iface = h2.get_interface(network_name=net_h2_name)
    h2_os_iface.ip_addr_add(addr=h2_addr, subnet=h2_subnet)
    h2_config_thread = h2.execute_thread(host_config_script)
    config_threads[h2] = h2_config_thread

    h3 = slice.get_node(name=h3_name)
    #if type(ip_address(h3.get_management_ip())) is IPv4Address:
    h3.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h3_os_iface = h3.get_interface(network_name=net_h3_name)
    h3_os_iface.ip_addr_add(addr=h3_addr, subnet=h3_subnet)
    h3_config_thread = h3.execute_thread(host_config_script)
    config_threads[h3] = h3_config_thread
    print(config_threads)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [ ]:
# Configure Switches , Setup P4 Docker
try:
    s1 = slice.get_node(name=s1_name)
    s1_h1_os_iface = s1.get_interface(network_name=net_h1_name)
    s1_s2_os_iface = s1.get_interface(network_name=net_s1_s2_name)
    s1_s3_os_iface = s1.get_interface(network_name=net_s1_s3_name)
        
    if type(ip_address(s1.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
    
    file_attributes = s1.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s1_h1_os_iface.get_os_interface()} {s1_s2_os_iface.get_os_interface()} {s1_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'"
    print("command = ",command)
    s1_config_thread = s1.execute_thread(command)
    config_threads[s1] = s1_config_thread


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [ ]:
try:
    s2 = slice.get_node(name=s2_name)
    s2_h2_os_iface = s2.get_interface(network_name=net_h2_name)
    s2_s1_os_iface = s2.get_interface(network_name=net_s1_s2_name)
    s2_s3_os_iface = s2.get_interface(network_name=net_s2_s3_name)

    if type(ip_address(s2.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
    
    file_attributes = s2.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command= f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s2_h2_os_iface.get_os_interface()} {s2_s1_os_iface.get_os_interface()} {s2_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'"
    print("command = ",command)
    s2_config_thread = s2.execute_thread(command)
    config_threads[s2] = s2_config_thread

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [ ]:
try:
    s3 = slice.get_node(name=s3_name)
    s3_h3_os_iface = s3.get_interface(network_name=net_h3_name)
    s3_s1_os_iface = s3.get_interface(network_name=net_s1_s3_name)
    s3_s2_os_iface = s3.get_interface(network_name=net_s2_s3_name)

    if type(ip_address(s3.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
     
    file_attributes = s3.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command = f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s3_h3_os_iface.get_os_interface()} {s3_s1_os_iface.get_os_interface()} {s3_s2_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
    print("command = ",command)
    s3_config_thread = s3.execute_thread(command)
    config_threads[s3] = s3_config_thread


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [ ]:
try:
    for node, thread in config_threads.items():
        print("Node, thread ",node,thread)
        print("\n\n")
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        print(f"stdout: {stdout}")
        print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [ ]:
# Confgure P4 Switch Tables
for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}_commands.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))
    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

In [ ]:
h2.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h1.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

In [ ]:
h1.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h2.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

In [ ]:
h3.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h3.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

In [ ]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 2 10.10.2.2 "message100"\'')

In [ ]:
print(h2.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

In [3]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta


slice_name = 'Nagm_P4Test01'
#Set end host to now plus 2 day
end_date = (datetime.now(timezone.utc) + timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")